In [0]:
from google.colab import drive
drive.mount('/content/drive')
import skimage
import os
import sys
import random
import math
import re
import time
import numpy as np
import cv2
import matplotlib
import matplotlib.pyplot as plt
import json
import pandas as pd
from PIL import Image
import torch
import torchvision
import torchvision.transforms.functional as TF
import torch.nn.functional as F
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.init as init
import torch.optim as optim
from torch.autograd import Variable
import tqdm
from tqdm import tqdm_notebook
os.chdir('/content/drive/My Drive/amit sethi')

ROOT_DIR = os.path.abspath("../../")
sys.path.append(ROOT_DIR)

# Data Path
TEST_PATH = 'Datasets/test/'
TRAIN_PATH = 'Datasets/nucleus/'

# Get train and test IDs
test_ids = next(os.walk(TEST_PATH))[1]
train_ids = next(os.walk(TRAIN_PATH))[1]

%matplotlib inline 

# Root directory of the project
torch.cuda.empty_cache()
# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "/content/drive/My Drive/amit sethi/logs")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
#### Defining Dataset ####

class NucleusDataset(torch.utils.data.Dataset):
  def __init__(self, images, masks, train = True):
    
    self.images = images
    self.masks = masks
    self.train = train


  def transforms(self, image, mask):
    
    if random.random() > 0.5:
      image = TF.hflip(image)
      mask = TF.hflip(mask)

    if random.random() > 0.5:
      image = TF.hflip(image)
      mask = TF.hflip(mask)

    if random.random() > 0.8:
      image = TF.affine(image, 90, (100,100), 1.0, 45.0)
      mask = TF.affine(mask, 90, (100,100), 1.0, 45.0)

    image = TF.to_tensor(image)
    mask = TF.to_tensor(mask)

    return image,mask


  def __len__(self):
    return len(self.images)

  def __getitem__(self,index): 

    image = self.images[index]
    mask = self.masks[index]

    if self.train:
      x,y = self.transforms(image, mask)

    else:
      x = TF.to_tensor(image)
      y = TF.to_tensor(mask)

    return x,y

In [0]:
def weights_init_normal(m):
    classname = m.__class__.__name__
    #print(classname)
    if classname.find('Conv') != -1:
        init.normal(m.weight.data, 0.0, 0.02)
    elif classname.find('Linear') != -1:
        init.normal(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        init.normal(m.weight.data, 1.0, 0.02)
        init.constant(m.bias.data, 0.0)


def weights_init_xavier(m):
    classname = m.__class__.__name__
    #print(classname)
    if classname.find('Conv') != -1:
        init.xavier_normal(m.weight.data, gain=1)
    elif classname.find('Linear') != -1:
        init.xavier_normal(m.weight.data, gain=1)
    elif classname.find('BatchNorm') != -1:
        init.normal(m.weight.data, 1.0, 0.02)
        init.constant(m.bias.data, 0.0)


def weights_init_kaiming(m):
    classname = m.__class__.__name__
    #print(classname)
    if classname.find('Conv') != -1:
        init.kaiming_normal_(m.weight.data, a=0, mode='fan_in')
    elif classname.find('Linear') != -1:
        init.kaiming_normal_(m.weight.data, a=0, mode='fan_in')
    elif classname.find('BatchNorm') != -1:
        init.normal_(m.weight.data, 1.0, 0.02)
        init.constant_(m.bias.data, 0.0)


def weights_init_orthogonal(m):
    classname = m.__class__.__name__
    #print(classname)
    if classname.find('Conv') != -1:
        init.orthogonal(m.weight.data, gain=1)
    elif classname.find('Linear') != -1:
        init.orthogonal(m.weight.data, gain=1)
    elif classname.find('BatchNorm') != -1:
        init.normal(m.weight.data, 1.0, 0.02)
        init.constant(m.bias.data, 0.0)


def init_weights(net, init_type='normal'):
    #print('initialization method [%s]' % init_type)
    if init_type == 'normal':
        net.apply(weights_init_normal)
    elif init_type == 'xavier':
        net.apply(weights_init_xavier)
    elif init_type == 'kaiming':
        net.apply(weights_init_kaiming)
    elif init_type == 'orthogonal':
        net.apply(weights_init_orthogonal)
    else:
        raise NotImplementedError('initialization method [%s] is not implemented' % init_type)


class _GridAttentionBlockND(nn.Module):
    def __init__(self, in_channels, gating_channels, inter_channels=None, dimension=3,
                 sub_sample_factor=(2,2,2)):
        super(_GridAttentionBlockND, self).__init__()

        assert dimension in [2, 3]

        # Downsampling rate for the input featuremap
        if isinstance(sub_sample_factor, tuple): self.sub_sample_factor = sub_sample_factor
        elif isinstance(sub_sample_factor, list): self.sub_sample_factor = tuple(sub_sample_factor)
        else: self.sub_sample_factor = tuple([sub_sample_factor]) * dimension

        # Default parameter set
        self.dimension = dimension
        self.sub_sample_kernel_size = self.sub_sample_factor

        # Number of channels (pixel dimensions)
        self.in_channels = in_channels
        self.gating_channels = gating_channels
        self.inter_channels = inter_channels

        if self.inter_channels is None:
            self.inter_channels = in_channels // 2
            if self.inter_channels == 0:
                self.inter_channels = 1

        if dimension == 3:
            conv_nd = nn.Conv3d
            bn = nn.BatchNorm3d
            self.upsample_mode = 'trilinear'
        elif dimension == 2:
            conv_nd = nn.Conv2d
            bn = nn.BatchNorm2d
            self.upsample_mode = 'bilinear'
        else:
            raise NotImplemented

        # Output transform
        self.W = nn.Sequential(
            conv_nd(in_channels=self.in_channels, out_channels=self.in_channels, kernel_size=1, stride=1, padding=0),
            bn(self.in_channels),
        )

        # Theta^T * x_ij + Phi^T * gating_signal + bias
        self.theta = conv_nd(in_channels=self.in_channels, out_channels=self.inter_channels,
                             kernel_size=self.sub_sample_kernel_size, stride=self.sub_sample_factor, padding=0, bias=False)
        self.phi = conv_nd(in_channels=self.gating_channels, out_channels=self.inter_channels,
                           kernel_size=1, stride=1, padding=0, bias=True)
        self.psi = conv_nd(in_channels=self.inter_channels, out_channels=1, kernel_size=1, stride=1, padding=0, bias=True)

        # Initialise weights
        for m in self.children():
            init_weights(m, init_type='kaiming')

        # Define the operation
        self.operation_function = self._concatenation



    def forward(self, x, g):
        '''
        :param x: (b, c, t, h, w)
        :param g: (b, g_d)
        :return:
        '''
        output = self.operation_function(x, g)
        return output

    def _concatenation(self, x, g):
        input_size = x.size()
        batch_size = input_size[0]
        assert batch_size == g.size(0)

        # theta => (b, c, t, h, w) -> (b, i_c, t, h, w) -> (b, i_c, thw)
        # phi   => (b, g_d) -> (b, i_c)
        theta_x = self.theta(x)
        theta_x_size = theta_x.size()

        # g (b, c, t', h', w') -> phi_g (b, i_c, t', h', w')
        #  Relu(theta_x + phi_g + bias) -> f = (b, i_c, thw) -> (b, i_c, t/s1, h/s2, w/s3)
        phi_g = F.upsample(self.phi(g), size=theta_x_size[2:], mode=self.upsample_mode)
        f = F.relu(theta_x + phi_g, inplace=True)

        #  psi^T * f -> (b, psi_i_c, t/s1, h/s2, w/s3)
        sigm_psi_f = F.sigmoid(self.psi(f))

        # upsample the attentions and multiply
        sigm_psi_f = F.upsample(sigm_psi_f, size=input_size[2:], mode=self.upsample_mode)
        y = sigm_psi_f.expand_as(x) * x
        W_y = self.W(y)

        return W_y, sigm_psi_f


class GridAttentionBlock2D(_GridAttentionBlockND):
    def __init__(self, in_channels, gating_channels, inter_channels=None,
                 sub_sample_factor=(2,2)):
        super(GridAttentionBlock2D, self).__init__(in_channels,
                                                   inter_channels=inter_channels,
                                                   gating_channels=gating_channels,
                                                   dimension=2,
                                                   sub_sample_factor=sub_sample_factor,
                                                   )


In [0]:
### utility functions ###

def train_creator(ids, root):
  images = []
  masks = []

  for i in range(len(ids)):

    image_name = root + ids[i] + '/images/' + ids[i] + '.png'
    mask_name = root + ids[i] + '/masks/' + ids[i] + '.png'
    original_image = Image.open(image_name)
    original_mask = Image.open(mask_name)

    width, height = original_image.size
    for i in range(0,height,200):
      for j in range(0,width,200):
        box = (j,i,j+200, i+200)
        image = original_image.crop(box)
        mask = original_mask.crop(box)
        images.append(image)
        masks.append(mask)
    
  return images, masks

def predict(model, dataloader):
  image, masks = next(iter(dataloader))
  length = len(dataloader)
  outputs = model(image.cuda())

  return image,outputs, masks

def show(img):
    npimg = img.detach().numpy()
    plt.imshow(np.transpose(npimg, (1,2,0)), interpolation='nearest')

In [0]:
### Defining UNET Architecture ###
class unet(nn.Module):

  def contracting_block(self, in_channels, out_channels):
    block = nn.Sequential(nn.Conv2d(in_channels,out_channels, 3, padding = 1), nn.ReLU(), nn.BatchNorm2d(out_channels),
                          nn.Conv2d(out_channels, out_channels, 3, padding  = 1), nn.ReLU(), nn.BatchNorm2d(out_channels))
    
    return block

  def expansive_block(self, in_channels, mid_channel, out_channels):
    block = nn.Sequential(nn.Conv2d(in_channels, mid_channel, 3, padding = 1), nn.ReLU(), nn.BatchNorm2d(mid_channel),
                          nn.Conv2d(mid_channel, mid_channel, 3, padding = 1), nn.ReLU(), nn.BatchNorm2d(mid_channel),
                          nn.ConvTranspose2d(mid_channel, out_channels, 3, 2,padding = 1,output_padding = 1))
    return block

  def final_block(self, in_channels, mid_channel, out_channels):
    block = nn.Sequential(nn.Conv2d(in_channels, mid_channel, 3, padding = 1), nn.ReLU(), nn.BatchNorm2d(mid_channel),
                          nn.Conv2d(mid_channel, mid_channel, 3, padding = 1), nn.ReLU(), nn.BatchNorm2d(mid_channel), 
                          nn.Conv2d(mid_channel, out_channels, 3, padding =1), nn.Sigmoid())
    return block

  def __init__(self, in_channel, out_channel):
    super(unet, self).__init__()

    self.encode1 = self.contracting_block(in_channel, 64)
    self.maxpool1 = nn.MaxPool2d(kernel_size = 2)
    self.encode2 = self.contracting_block(64,128)
    self.maxpool2 = nn.MaxPool2d(2)
    self.encode3 = self.contracting_block(128,256)
    self.maxpool3 = nn.MaxPool2d(2)

    self.bottleneck = self.expansive_block(256,512,256)

    self.decode3 = self.expansive_block(512,256,128)
    self.decode2 = self.expansive_block(256,128,64)

    self.ag1 = GridAttentionBlock2D(256,256)
    self.ag2 = GridAttentionBlock2D(128,128)
    self.ag3 = GridAttentionBlock2D(64,64)
    
    self.final_layer = self.final_block(128,64,out_channel)

  
  def crop_and_concat(self, upsampled, bypass):
      c = (bypass.size()[2] - upsampled.size()[2]) // 2
      bypass = F.pad(bypass, (-c, -c, -c, -c))
      return torch.cat((upsampled, bypass), 1)


  def forward(self,x):
    encode1 = self.encode1(x)
    maxpool1 = self.maxpool1(encode1)
    encode2 = self.encode2(maxpool1)
    maxpool2 = self.maxpool2(encode2)
    encode3 = self.encode3(maxpool2)
    maxpool3 = self.maxpool3(encode3)

    bottleneck = self.bottleneck(maxpool3)

    gate1 = self.ag1(encode3, bottleneck)
    encode3 = encode3 + gate1[0]

    decode3 = self.crop_and_concat(bottleneck, encode3)
    cat_layer2 = self.decode3(decode3)
    
    gate2 = self.ag2(encode2, cat_layer2)
    encode2 = encode2 + gate2[0]

    decode2 = self.crop_and_concat(cat_layer2, encode2)
    cat_layer1 = self.decode2(decode2)
    
    gate3 = self.ag3(encode1, cat_layer1)
    encode1 = encode1 + gate3[0]

    decode1 = self.crop_and_concat(cat_layer1, encode1)
    final = self.final_layer(decode1)

    return final

In [0]:
train_images, train_masks = train_creator(train_ids, TRAIN_PATH)
test_images, test_masks = train_creator(test_ids, TEST_PATH)

train_dataset = NucleusDataset(train_images, train_masks, True)
lengths = [int(len(train_images) * 0.8), int(len(train_images) * 0.2)]
train_dataset, val_dataset = torch.utils.data.random_split(train_dataset, lengths)
test_dataset = NucleusDataset(test_images, test_masks, False)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size = 5, shuffle = True, num_workers = 2)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size = 5, shuffle = False, num_workers = 2)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size = 25, shuffle = False, num_workers = 2)

In [0]:
### Training Functions ###

def train(dataloader, model, optimizer, criterion):
  model.train()
  train_losses.append(0)
  progbar = tqdm_notebook(total = len(dataloader), desc = 'UNet Train')


  for i, (inputs,targets) in enumerate(dataloader):
    optimizer.zero_grad()
    
    inputs, targets = Variable(inputs.cuda()), Variable(targets.cuda())

    outputs = model(inputs)
    error = criterion(outputs, targets)

    error.backward()
    optimizer.step()

    train_losses[-1] = train_losses[-1] + error.data
    progbar.set_description('Train (loss=%.4f)' % (train_losses[-1]/(i+1)))
    progbar.update(1)

  train_losses[-1] = train_losses[-1]/len(dataloader)


def val(dataloader, model, criterion):
  
  global best_loss
  progbar = tqdm_notebook(total = len(dataloader), desc = 'UNet Val')
  model.eval()

  val_losses.append(0)

  for i, (inputs, targets) in enumerate(dataloader):
    inputs, targets = Variable(inputs.cuda()), Variable(targets.cuda())

    outputs = model(inputs)
    error = criterion(outputs, targets)

    val_losses[-1] = val_losses[-1] + error.data
    progbar.set_description('Val (loss = %.4f)' % (train_losses[-1] /(i+1)))
    progbar.update(1)

  val_losses[-1] = val_losses[-1]/(len(dataloader))

  if best_loss > val_losses[-1]:
    best_loss = val_losses[-1]
    print('SAVING....')
    state = {'model' : model}

    torch.save(state, 'monuseg_unet_attention' + '.ckpt.t7')

In [0]:
model = unet(3,1)
model = model.cuda()

criterion = torch.nn.BCELoss()

train_losses = []
val_losses = []

epochs = 100

lrs = [1e-2, 1e-3, 1e-4, 1e-4, 1e-4]

In [0]:
progbar_1 = tqdm_notebook(total = 100, desc = 'epochs')
best_loss = 1e5

for epoch in range(100):

  if epoch == 0:
    optimizer = torch.optim.SGD(model.parameters(), lr = lrs[0], momentum = 0.9, weight_decay= 5e-4)

  if epoch == 20:
    optimizer = torch.optim.SGD(model.parameters(), lr = lrs[1], momentum = 0.9, weight_decay= 5e-4)

  if epoch == 40:
    optimizer = torch.optim.SGD(model.parameters(), lr = lrs[2], momentum = 0.9, weight_decay= 5e-4)

  if epoch == 60:
    optimizer = torch.optim.SGD(model.parameters(), lr = lrs[3], momentum = 0.9, weight_decay= 5e-4)

  train(train_loader, model, optimizer, criterion)
  val(val_loader, model, criterion)

  progbar_1.update(1)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """Entry point for launching an IPython kernel.


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:2796: UserWarning: nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.")
/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:2973: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))
/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1569: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:29: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook

SAVING....


/usr/local/lib/python3.6/dist-packages/torch/serialization.py:402: UserWarning: Couldn't retrieve source code for container of type unet. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:402: UserWarning: Couldn't retrieve source code for container of type GridAttentionBlock2D. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


...

Exception in thread Thread-4:
Traceback (most recent call last):
  File "/usr/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/usr/local/lib/python3.6/dist-packages/tqdm/_monitor.py", line 78, in run
    instances = self.get_instances()
  File "/usr/local/lib/python3.6/dist-packages/tqdm/_monitor.py", line 58, in get_instances
    return [i for i in self.tqdm_cls._instances.copy()
  File "/usr/lib/python3.6/_weakrefset.py", line 92, in copy
    return self.__class__(self)
  File "/usr/lib/python3.6/_weakrefset.py", line 50, in __init__
    self.update(data)
  File "/usr/lib/python3.6/_weakrefset.py", line 119, in update
    for element in other:
  File "/usr/lib/python3.6/_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration



SAVING....


SAVING....


SAVING....


SAVING....


In [0]:
checkpoints = torch.load('monuseg_unet1.ckpt.t7')
model = checkpoints['model']

checkpoints = torch.load('monuseg_hrnet1.ckpt.t7')
model1 = checkpoints['model']

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:657: SourceChangeWarning: source code of class 'torch.nn.modules.conv.Conv2d' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:657: SourceChangeWarning: source code of class 'torch.nn.modules.activation.ReLU' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:657: SourceChangeWarning: source code of class 'torch.nn.modules.conv.ConvTranspose2d' has changed. you can retrieve the original source code by accessing the object's source attribute or set 

AttributeError: ignored

In [0]:
inputs, outputs, masks = predict(model, test_loader)
inputs1, outputs1, masks1 = predict(model1, test_loader)

In [0]:
show(torchvision.utils.make_grid(inputs, 5))

In [0]:
show(torchvision.utils.make_grid(outputs.cpu(), 5))

In [0]:
show(torchvision.utils.make_grid(masks, 5))